# ActBlue donor profile

In [1]:
import json
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import os

In [2]:
%matplotlib inline

plt.style.use('ggplot')

pd.options.display.float_format = '{:,.2f}'.format

In [3]:
def read_or_save(name, func):
    path = 'pickles/' + name + '.pickle.gz'
    if (os.path.isfile(path)):
        return pd.read_pickle(path)
    else:
        result = func()
        os.makedirs('pickles', exist_ok=True)
        result.to_pickle(path)
        return result

In [4]:
committee_id = "C00401224"
year_since = 2013

### Connect to the PostgreSQL database

In [5]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf['dbname'],
    user=conf['user'],
    host=conf['host'],
    password=conf['password']
)

### Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?

In [6]:
actblue_states = read_or_save(
    'actblue_states',
    lambda: pd.read_sql("""
    SELECT contributor_state,
           count(*),
           sum(contribution_amount)
    FROM fec_contributions
    WHERE filing_id IN
        (SELECT filing_id
         FROM fec_pac_summaries
         JOIN fec_amended_filings USING (filing_id)
         WHERE filer_committee_id_number = '""" + committee_id + """'
           AND extract(YEAR
                       FROM coverage_through_date) > """ + str(year_since) + """
         ORDER BY coverage_through_date DESC)
      AND form_type = 'SA11AI'
    GROUP BY contributor_state
    """, con=conn)
)
actblue_states.sort_values(by=['sum'], ascending=False)

,contributor_state,count,sum
61,CA,12775426,"304,716,691.17"
22,NY,5935713,"169,544,004.50"
103,MA,2925772,"78,702,360.92"
68,TX,3371982,"73,030,816.37"
35,FL,3340947,"69,506,598.44"
98,IL,2587780,"58,333,363.93"
102,WA,2891177,"54,541,566.29"
7,VA,1730251,"48,037,247.79"
92,PA,2281919,"47,828,105.60"
94,MD,1550863,"42,973,364.00"


In [7]:
clinton_votes = pd.read_csv('clintonvotes.csv')

clinton_votes

votes_vs_contribs = actblue_states.merge(clinton_votes, left_on='contributor_state', right_on='postal', how='inner')

votes_vs_contribs['dollars_per'] = votes_vs_contribs['sum']/votes_vs_contribs['votes']

votes_vs_contribs.sort_values(by=['dollars_per'], ascending=False)

,contributor_state,count,sum,state,postal,votes,dollars_per
34,DC,503502,"32,216,622.44",District of Columbia,DC,282830,113.91
1,VT,515078,"9,842,647.08",Vermont,VT,178573,55.12
21,MT,333586,"8,055,215.20",Montana,MT,177709,45.33
32,NM,824161,"15,198,396.94",New Mexico,NM,385234,39.45
49,MA,2925772,"78,702,360.92",Massachusetts,MA,1995196,39.45
9,NY,5935713,"169,544,004.50",New York,NY,4556142,37.21
44,AK,208959,"4,101,780.71",Alaska,AK,116454,35.22
27,CA,12775426,"304,716,691.17",California,CA,8753792,34.81
28,NH,505394,"11,690,132.38",New Hampshire,NH,348526,33.54
19,WY,79040,"1,803,137.86",Wyoming,WY,55973,32.21


In [8]:
votes_vs_contribs['contribs_per'] = votes_vs_contribs['count']/votes_vs_contribs['votes']

votes_vs_contribs.sort_values(by=['contribs_per'], ascending=False)

,contributor_state,count,sum,state,postal,votes,dollars_per,contribs_per
1,VT,515078,"9,842,647.08",Vermont,VT,178573,55.12,2.88
32,NM,824161,"15,198,396.94",New Mexico,NM,385234,39.45,2.14
29,OR,2005634,"30,653,845.56",Oregon,OR,1002106,30.59,2.00
21,MT,333586,"8,055,215.20",Montana,MT,177709,45.33,1.88
44,AK,208959,"4,101,780.71",Alaska,AK,116454,35.22,1.79
34,DC,503502,"32,216,622.44",District of Columbia,DC,282830,113.91,1.78
48,WA,2891177,"54,541,566.29",Washington,WA,1742718,31.30,1.66
49,MA,2925772,"78,702,360.92",Massachusetts,MA,1995196,39.45,1.47
27,CA,12775426,"304,716,691.17",California,CA,8753792,34.81,1.46
28,NH,505394,"11,690,132.38",New Hampshire,NH,348526,33.54,1.45


### Questions to answer
> - Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?
> - Where is the money going? Are people giving to local candidates or are they giving to candidates around the country? Which campaigns/candidates/groups have done the best job tapping into this network?
> - What’s the gender breakdown and how does it compare to the gender breakdown of itemized contributions? Is it different?
> - Are these new donors?
> - Can we tell if these are recurring contributions or one time contributions?
> - Do the majority of these donors give repeatedly or are they one and done? How does that compare to itemized contributions? How “sticky” is this?


In [9]:
actblue_in_state = read_or_save(
    'actblue_in_state',
    lambda: pd.read_sql("""
    SELECT CASE
               WHEN beneficiary_candidate_state = contributor_state THEN TRUE
               ELSE FALSE
           END AS in_state,
           sum(contribution_amount),
           count(*)
    FROM fec_contributions
    JOIN fec_expenditures ON fec_contributions.filing_id IN
      (SELECT filing_id
       FROM fec_pac_summaries
       JOIN fec_amended_filings USING (filing_id)
       WHERE filer_committee_id_number = '""" + committee_id + """'
         AND extract(YEAR
                     FROM coverage_through_date) > 2016
       ORDER BY coverage_through_date DESC)
    AND fec_expenditures.filing_id = fec_contributions.filing_id
    AND fec_contributions.form_type = 'SA11AI'
    AND fec_expenditures.form_type = 'SB23'
    AND replace(transaction_id_number,'SB23_','') = replace(transaction_id,'SA11AI_','')
    AND beneficiary_candidate_state IS NOT NULL
    GROUP BY in_state
    """, con=conn)
)
actblue_in_state

,in_state,sum,count
0,False,"202,671,017.32",6905475
1,True,"141,932,982.22",2580001


In [10]:
actblue_freq = read_or_save(
    'actblue_freq',
    lambda: pd.read_sql("""
    SELECT COUNT,
           count(*) AS count_of_count
    FROM
      (SELECT contributor_first_name,
              contributor_last_name,
              left(contributor_zip_code,5),
              count(*) AS COUNT
       FROM fec_contributions
       WHERE filing_id IN
           (SELECT filing_id
            FROM fec_pac_summaries
            JOIN fec_amended_filings USING (filing_id)
            WHERE filer_committee_id_number = '""" + committee_id + """'
              AND extract(YEAR
                          FROM coverage_through_date) > 2016
            ORDER BY coverage_through_date DESC)
         AND form_type = 'SA11AI'
       GROUP BY contributor_first_name,
                contributor_last_name,
                left(contributor_zip_code,5)) AS donors
    GROUP BY COUNT
    """, con=conn)
)
actblue_freq

,count,count_of_count
0,652,4
1,273,48
2,51,2414
3,1091,2
4,951,2
5,839,1
6,70,1293
7,1075,2
8,350,28
9,539,3


In [11]:
actblue_addicts = read_or_save(
    'actblue_addicts',
    lambda: pd.read_sql("""
    SELECT COUNT,
           count(*) AS count_of_count,
           array_agg(DISTINCT contributor_organization_name),
           array_agg(DISTINCT contributor_first_name),
           array_agg(DISTINCT contributor_last_name),
           array_agg(DISTINCT contributor_zip_code)
    FROM
      (SELECT contributor_organization_name,
              contributor_first_name,
              contributor_last_name,
              left(contributor_zip_code,5) as contributor_zip_code,
              count(*) AS COUNT
       FROM fec_contributions
       WHERE filing_id IN
           (SELECT filing_id
            FROM fec_pac_summaries
            JOIN fec_amended_filings USING (filing_id)
            WHERE filer_committee_id_number = '""" + committee_id + """'
              AND extract(YEAR
                          FROM coverage_through_date) > 2016
            ORDER BY coverage_through_date DESC)
         AND form_type = 'SA11AI'
       GROUP BY contributor_organization_name,
                contributor_first_name,
                contributor_last_name,
                left(contributor_zip_code,5)) AS donors
    GROUP BY COUNT
    HAVING count(*) <= 3
    ORDER BY COUNT DESC
    LIMIT 30
    """, con=conn)
)
actblue_addicts



,count,count_of_count,array_agg,array_agg,array_agg,array_agg
0,13452,1,[None],[SIBYLLE],[BARLOW],[01742]
1,8807,1,[None],[TERRY],[WOLFE],[26508]
2,8401,1,[None],[RICHARD],[GOLDSTEIN],[11962]
3,7969,1,[None],[JOHN],[COMELLA],[19103]
4,7659,1,[None],[VICKI],[FARRAR],[83714]
5,7267,1,[None],[CAROLINE],[MERRIAM],[20007]
6,7216,1,[None],[WILLIAM],[CHEEK],[92115]
7,7131,1,[None],[ROXANNE],[WARREN],[10025]
8,6442,1,[None],[LUCY],[HARMON],[75771]
9,6234,1,[None],[MARTHA],[UTZ],[95120]


### How candidates in the DCCC’s Red to Blue program do via ActBlue.

### Where do contributors give from?

### Whether candidates who pledge not to take corporate PAC money are getting more contributions than other candidates.

### More examples of ActBlue addicts.

### Spreadsheet of the 1,307 candidate committees that got at least some money through ActBlue this cycle.

### Unitemized contributions versus itemized for this point in the cycle for both the 2018 and 2014 cycle, and break it down by Republican versus Democrat?